# Integrated Bioinformatics Project : Flux_Balance_F_Prausnitzii 
#### Students: Michael Shawn Neilsson, Biancamaria FLorenzi, Prabhat Juyal, Tim Blokker
#### Supervisors: Clémence Joseph, Karoline Faust

In [1]:
#import packages
from libchebipy import ChebiEntity
import libchebipy
import pubchempy as pcp
import io
import requests
import json
import cobra
import functions
from functions import *

In [2]:
#model = cobra.io.read_sbml_model('Data/f_prausnitzii.xml')
#info(model)

In [3]:
model=cobra.io.load_matlab_model('Data/iFpraus_v_1_0.mat')
info(model)

Metabolites :  833
Reactions :  1030
Genes :  602
Compartments :  {'c': '', 'e': ''}
Objective function : 
 Maximize
1.0*Biomass_FP - 1.0*Biomass_FP_reverse_ee33b 



In [4]:
#https://www.thielelab.eu/in-silico-models
#model = cobra.io.read_sbml_model('Data/iFpraus_v_1_0.xml')
#(model2,errors)=(cobra.io.sbml.validate_sbml_model('Data/iFpraus_v_1_0.xml'))
#print(errors)

In [5]:
outputmodel= cobra.io.save_json_model(model, "Data/iFpraus.json")

### What are the different reactions
e: extracellular

c: cytosol

EX_ : exchange reaction
### Gifu Anaerobic Medium (mGAM) from:
https://hyserve.com/files/05433_GAM-Broth_Modified_final.pdf


- Peptone, Soya Peptone, Proteose Peptone: source of amino acids -> most likely all of them 
- Yeast extract, liver extract, meat extract, digested serum -> amino acids but also a lot of other crap, will leave this till the end 
- Dextrose -> same as glucose (https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:17634) and so -> glc_D
- Soluble starch -> starch1200 (potatoe starch)
- L-Tryptophane -> trp__L
- L-Cystein Hydrochloride -> cys_L and cl and h (hydrochloride<-> HCL becomes cl and h)
- Sodium Thioglycolate -> thiog and na1
- L-Arginine -> arg_L
- Vitamine K1 -> phllqne https://pubchem.ncbi.nlm.nih.gov/compound/Vitamin-K1#section=Depositor-Supplied-Synonyms 
- Hemin -> Iron and Chlorine  fe3 and cl  and perhaps ppp9 https://pubchem.ncbi.nlm.nih.gov/#query=3-%5B18-(2-carboxyethyl)-8%2C13-bis(ethenyl)-3%2C7%2C12%2C17-tetramethylporphyrin-21%2C23-diid-2-yl%5Dpropanoic%20acid%3Biron(3%2B)%3Bchloride 
- Potassium Dihydrogen Phosphate -> pi, k, h
- Sodium Chloride -> na1 cl

Digested serum : https://www.nebiolabs.com.au/-/media/catalog/datacards-or-manuals/p8108datasheet-lot0021306.pdf	

Generally useful page ; https://pubchem.ncbi.nlm.nih.gov/#query=Hemin

check also: https://opencobra.github.io/cobratoolbox/latest/tutorials/tutorialMetabotoolsI.html 

In [6]:
import numpy as np
aa=("ala,arg,asn,asp,asx,cys,glu,gln,glx,gly,his,ile,leu,lys,met,phe,pro,ser,thr,trp,tyr,val").split(",")
aa=[aminoacid +"_L(e)" for aminoacid in aa]
print(aa)

['ala_L(e)', 'arg_L(e)', 'asn_L(e)', 'asp_L(e)', 'asx_L(e)', 'cys_L(e)', 'glu_L(e)', 'gln_L(e)', 'glx_L(e)', 'gly_L(e)', 'his_L(e)', 'ile_L(e)', 'leu_L(e)', 'lys_L(e)', 'met_L(e)', 'phe_L(e)', 'pro_L(e)', 'ser_L(e)', 'thr_L(e)', 'trp_L(e)', 'tyr_L(e)', 'val_L(e)']


In [7]:
model1=model.copy()
print("\nModel Medium")
#model1.objective = {model1.reactions.get_by_id('EX_ac(e)'): 1}
model1.objective = {model1.reactions.get_by_id('Biomass_FP'): 1}

print(model1.summary())


Model Medium
Objective
1.0 Biomass_FP = 69.87885590787737

Uptake
------
    Metabolite          Reaction   Flux  C-Number  C-Flux
    Lcystin[e]     EX_Lcystin(e)  3.502         6   0.05%
        ade[e]         EX_ade(e)  864.9         5  10.85%
     adocbl[e]      EX_adocbl(e) 0.3929        72   0.07%
     alaala[e]      EX_alaala(e)  6.462         6   0.10%
     alagln[e]      EX_alagln(e)  30.81         8   0.62%
      arg-L[e]       EX_arg_L(e)   1000         6  15.05%
      asn-L[e]       EX_asn_L(e)   16.4         4   0.16%
        btn[e]         EX_btn(e) 0.3929        10   0.01%
        ca2[e]         EX_ca2(e) 0.3929         0   0.00%
         cl[e]          EX_cl(e) 0.3929         0   0.00%
    cobalt2[e]     EX_cobalt2(e) 0.3929         0   0.00%
     cps_fp[e]      EX_cps_fp(e) 0.3929        63   0.06%
        cu2[e]         EX_cu2(e) 0.3929         0   0.00%
        fe3[e]         EX_fe3(e) 0.7858         0   0.00%
        fol[e]         EX_fol(e)  1.179        19   0.06

In [8]:
#use for later perhaps 

#modelMedium = setMediumConstraints(starting_model, set_inf,
#current_inf,... medium_composition, met_Conc_mM, cellConc, t, cell-
#Weight, mediumCompounds,... mediumCompounds_lb);

#met_Conc_mM = [0.1;1.15;0.15;0.379;0.208;2;0.136;0.133;0.0968;0.382;0.382;0.274;0.101;0.0909;0.153;0.174;...
#0.286;0.168;0.0245;0.129;0.171;0.00863;0.00082;0.0214;0.000524;0.00227;0.082;0.00485;0.000532;0.00297;...
#0.194;0.424;0;5.33;23.81;127.26;5.63;11.11;0;0;0;1;0;0.00326;0.0073]
#customizedConstraints_lb = [-2.3460;0;0;-500;-100;-100;-100];
#customizedConstraints_ub = [500;0;0;500;500;500;500];

In [9]:
MediumMgam = ['ala_L_(e)', 'arg_L_(e)', 'asn_L_(e)', 'asp_L_(e)', 'asx_L_(e)', 'cys_L_(e)', 'glu_L_(e)', 'gln_L_(e)', 
              'glx_L_(e)', 'gly_L_(e)', 'his_L_(e)', 'ile_L_(e)', 'leu_L_(e)', 'lys_L_(e)', 'met_L_(e)', 'phe_L_(e)', 
              'pro_L_(e)', 'ser_L_(e)', 'thr_L_(e)', 'trp_L_(e)', 'tyr_L_(e)', 'val_L_(e)', 'glc_D_(e)', 'starch1200_(e)', 
              'cl_(e)', 'h_(e)',"ac_(e)", "so4_(e)" #'thiog_(e)'
              , 'na1_(e)', 'fe3_(e)', 'ppp9_(e)', 'h2o_(e)', 
              'pi_(e)', 'k_(e)']
MediumMgam=[mets[:-4] +"(e)" for mets in MediumMgam]
print(MediumMgam)
model2=model.copy()
model_mgam=medium(model2,MediumMgam)
print('\nMedium_mgam')
model_mgam.objective = {model_mgam.reactions.get_by_id('Biomass_FP'): 1}
#model_mgam.optimize()
print(model_mgam.summary())

['ala_L(e)', 'arg_L(e)', 'asn_L(e)', 'asp_L(e)', 'asx_L(e)', 'cys_L(e)', 'glu_L(e)', 'gln_L(e)', 'glx_L(e)', 'gly_L(e)', 'his_L(e)', 'ile_L(e)', 'leu_L(e)', 'lys_L(e)', 'met_L(e)', 'phe_L(e)', 'pro_L(e)', 'ser_L(e)', 'thr_L(e)', 'trp_L(e)', 'tyr_L(e)', 'val_L(e)', 'glc_D(e)', 'starch1200(e)', 'cl(e)', 'h(e)', 'ac(e)', 'so4(e)', 'na1(e)', 'fe3(e)', 'ppp9(e)', 'h2o(e)', 'pi(e)', 'k(e)']

Medium_mgam
Objective
1.0 Biomass_FP = -6.330253172891172e-16

Uptake
------
Metabolite     Reaction  Flux  C-Number  C-Flux
      h[e]      EX_h(e) 3.912         0   0.00%
    na1[e]    EX_na1(e)    10         0   0.00%
  ser-L[e]  EX_ser_L(e) 7.823         3  99.36%
   hpyr[c]    sink_hpyr  0.05         3   0.64%

Secretion
---------
Metabolite   Reaction   Flux  C-Number  C-Flux
    na1[c]     DM_NA1    -10         0   0.00%
     ac[e]   EX_ac(e)  -0.05         2   0.42%
    but[e]  EX_but(e) -3.912         4  66.24%
    co2[e]  EX_co2(e) -7.873         1  33.33%
    nh4[e]  EX_nh4(e) -7.823         0

In [10]:
MediumMgam = ['ala_L_(e)', 'arg_L_(e)', 'asn_L_(e)', 'asp_L_(e)', 'asx_L_(e)', 'cys_L_(e)', 'glu_L_(e)', 'gln_L_(e)', 
              'glx_L_(e)', 'gly_L_(e)', 'his_L_(e)', 'ile_L_(e)', 'leu_L_(e)', 'lys_L_(e)', 'met_L_(e)', 'phe_L_(e)', 
              'pro_L_(e)', 'ser_L_(e)', 'thr_L_(e)', 'trp_L_(e)', 'tyr_L_(e)', 'val_L_(e)', 'glc_D_(e)', 'starch1200_(e)', 
              'cl_(e)', 'h_(e)',"ac_(e)", "so4_(e)" #'thiog_(e)'
              , 'na1_(e)', 'fe3_(e)', 'ppp9_(e)', 'h2o_(e)', 
              'pi_(e)', 'k_(e)']
MediumMgam=[mets[:-4] +"(e)" for mets in MediumMgam]
print(MediumMgam)
model2=model.copy()
model_mgam=medium(model2,MediumMgam)
print('\nMedium_mgam')
for i in range(1,len(model_mgam.reactions)):
    model_mgam.reactions[i].upper_bound=1000
    model_mgam.reactions[i].lower_bound=-1000 #does not change till 77,78
model_mgam.objective = {model_mgam.reactions.get_by_id('Biomass_FP'): 1}
#model_mgam.optimize()
print(model_mgam.summary())

['ala_L(e)', 'arg_L(e)', 'asn_L(e)', 'asp_L(e)', 'asx_L(e)', 'cys_L(e)', 'glu_L(e)', 'gln_L(e)', 'glx_L(e)', 'gly_L(e)', 'his_L(e)', 'ile_L(e)', 'leu_L(e)', 'lys_L(e)', 'met_L(e)', 'phe_L(e)', 'pro_L(e)', 'ser_L(e)', 'thr_L(e)', 'trp_L(e)', 'tyr_L(e)', 'val_L(e)', 'glc_D(e)', 'starch1200(e)', 'cl(e)', 'h(e)', 'ac(e)', 'so4(e)', 'na1(e)', 'fe3(e)', 'ppp9(e)', 'h2o(e)', 'pi(e)', 'k(e)']

Medium_mgam
Objective
1.0 Biomass_FP = 131.51398638093758

Uptake
------
 Metabolite       Reaction   Flux  C-Number  C-Flux
    4hba[c]        DM_4HBA  64.77         7   4.63%
    3mop[e]     EX_3mop(e)  521.6         6  31.94%
   acglu[e]    EX_acglu(e)  57.79         7   4.13%
     ade[e]      EX_ade(e)  28.42         5   1.45%
  adocbl[e]   EX_adocbl(e) 0.7395        72   0.54%
  alaala[e]   EX_alaala(e)  38.38         6   2.35%
  alahis[e]   EX_alahis(e)  12.18         9   1.12%
  alaleu[e]   EX_alaleu(e)  57.69         9   5.30%
   arg-L[e]    EX_arg_L(e)  37.91         6   2.32%
     btn[e]      E

In [11]:
MediumMgam = ['ala_L_(e)', 'arg_L_(e)', 'asn_L_(e)', 'asp_L_(e)', 'asx_L_(e)', 'cys_L_(e)', 'glu_L_(e)', 'gln_L_(e)', 
              'glx_L_(e)', 'gly_L_(e)', 'his_L_(e)', 'ile_L_(e)', 'leu_L_(e)', 'lys_L_(e)', 'met_L_(e)', 'phe_L_(e)', 
              'pro_L_(e)', 'ser_L_(e)', 'thr_L_(e)', 'trp_L_(e)', 'tyr_L_(e)', 'val_L_(e)', 'glc_D_(e)', 'starch1200_(e)', 
              'cl_(e)', 'h_(e)',"ac_(e)", "so4_(e)" #'thiog_(e)'
              , 'na1_(e)', 'fe3_(e)', 'ppp9_(e)', 'h2o_(e)', 
              'pi_(e)', 'k_(e)']
MediumMgam=[mets[:-4] +"(e)" for mets in MediumMgam]
print(MediumMgam)
model2=model.copy()
model_mgam=medium(model2,MediumMgam)
print('\nMedium_mgam')
for i in range(1,len(model_mgam.reactions)):
    model_mgam.reactions[i].upper_bound=1000
    if model_mgam.reactions[i].lower_bound == 0:
        model_mgam.reactions[i].lower_bound=0 #does not change till 77,78
    else:
        model_mgam.reactions[i].lower_bound=-1000 #does not change till 77,78
mediums=model_mgam.medium
model_mgam.objective = {model_mgam.reactions.get_by_id('Biomass_FP'): 1}
print(model_mgam.summary())

['ala_L(e)', 'arg_L(e)', 'asn_L(e)', 'asp_L(e)', 'asx_L(e)', 'cys_L(e)', 'glu_L(e)', 'gln_L(e)', 'glx_L(e)', 'gly_L(e)', 'his_L(e)', 'ile_L(e)', 'leu_L(e)', 'lys_L(e)', 'met_L(e)', 'phe_L(e)', 'pro_L(e)', 'ser_L(e)', 'thr_L(e)', 'trp_L(e)', 'tyr_L(e)', 'val_L(e)', 'glc_D(e)', 'starch1200(e)', 'cl(e)', 'h(e)', 'ac(e)', 'so4(e)', 'na1(e)', 'fe3(e)', 'ppp9(e)', 'h2o(e)', 'pi(e)', 'k(e)']

Medium_mgam
Objective
1.0 Biomass_FP = 0.0

Uptake
------
Empty DataFrame
Columns: [Metabolite, Reaction, Flux, C-Number, C-Flux]
Index: []

Secretion
---------
Empty DataFrame
Columns: [Metabolite, Reaction, Flux, C-Number, C-Flux]
Index: []



In [12]:
fba = model.optimize()
fluxes = deleteNull(fba.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes),len(fluxes)/len(model.reactions)*100))
print(fba.objective_value)


Active reaction : 364 which represent 35.339806% of the model
69.8788559078775


In [13]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, max_growth)

EX_Lcystin(e)          3.501825
EX_ade(e)            864.852705
EX_adocbl(e)           0.392915
EX_alaala(e)           6.462117
EX_alagln(e)          30.813920
EX_arg_L(e)         1000.000000
EX_asn_L(e)           16.399170
EX_btn(e)              0.392915
EX_ca2(e)              0.392915
EX_cl(e)               0.392915
EX_cobalt2(e)          0.392915
EX_cps_fp(e)           0.392915
EX_cu2(e)              0.392915
EX_fe3(e)            520.613611
EX_fol(e)              1.178744
EX_fum(e)           1000.000000
EX_glu_L(e)         1000.000000
EX_gua(e)             16.254032
EX_h(e)              269.528716
EX_his_L(e)            6.472389
EX_ins(e)            751.818425
EX_k(e)                0.392915
EX_lac_D(e)           65.317421
EX_leuleu(e)          15.326529
EX_lys_L(e)           23.308092
EX_mal_L(e)         1000.000000
EX_metala(e)          10.591818
EX_metsox_S_L(e)       0.236805
EX_mg2(e)              0.392915
EX_na1(e)           1000.000000
EX_nac(e)              0.785830
EX_o2(e)

In [14]:
from cobra.medium import minimal_medium
max_growth = model_mgam.slim_optimize()
minimal_medium(model_mgam, max_growth)

Series([], dtype: float64)

---
## Reinforced Clostridial Medium (RCM)
http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=CM0149&org=53&c=UK&lang=EN

* Yeast Extract -> [lots of stuff in it, unsure of what's garbage](https://www.chemicalbook.com/ChemicalProductProperty_EN_CB9440339.htm#:~:text=Yeast%20extract%20has%20a%20protein,aromatic%20compounds%20and%20other%20components.)
    * Glutathione: gthrd (reduced version)
    * 18 AA's: see above
    * Dextran: glc__D (it's made of glucose)
    * Mannan: mannan
    * Trehalose: tre
    * B-vitamins: (Clemence mentioned vitamins being important so I'm putting them all in)
        * \1. Thiamin -> thm
        * \2. Riboflavin -> ribflv
        * \3. Niacin -> trp__L (made of tryptophan)
        * \5. Pantothenic Acid -> pnto__R
        * \6. Pyridoxine -> pydxn
        * \7. Biotin -> btn
        * \9. Folic Acid -> ... can't find anything
        * \12. Cobalamin -> b12 (or cbl1)
    * Biotin
* Peptone -> Sticking with the above logic, probably a source of all AAs
* Glucose -> Assuming D_Glucose: glc__D
* Soluble Starch -> starch1200
* Sodium Chloride -> na1, cl
* Sodium Acetate -> Assuming breaking down into sodium and acetate: ac, na1
* Cysteine Hydrochloride -> cys_L, cl, h (could also be cys_D? cys_L is more widely used)
* Agar: Agarose (70%) and Agaropectin (30%) [according to Wiki](https://en.wikipedia.org/wiki/Agar)
    * Agarose: D-galactose & 3,6-anhydro-L-galactopyranose
        * D-galactose: gal
        * 3,6-anhydro-L-galactopyranose: [similar structure & compositon to beta-D-allose](https://pubchem.ncbi.nlm.nih.gov/#query=CID67020466%20structure&tab=similarity)
            * D-Allose: all__D
    * Agaropectin: D-galactose, L-galactose, pyruvate, sulfate
        * D-galactose: gal
        * L-galactose: gal__L
        * Pyruvate: pyr
        * Sulfate: so4

| Component              | Concentration (g/L) | BiGG Metabolites                                                                          |
|------------------------|---------------------|-------------------------------------------------------------------------------------------|
| Yeast Extract          | 13.0                | (see all AA's), gthrd, glc\__D, mannan, tre, thm, ribflv, trp\__L, pnto\__R, pydxn, btn, b12 |
| Peptone                | 10.0                | (see all AA's)                                                                            |
| Glucose                | 5.0                 | glc_D                                                                                     |
| Soluble Starch         | 1.0                 | starch1200                                                                                |
| Sodium Chloride        | 5.0                 | na1, cl                                                                                   |
| Sodium Acetate         | 3.0                 | na1, ac                                                                                   |
| Cysteine Hydrochloride | 0.5                 | cys_L, cl, h                                                                              |
| Agar                   | 0.5                 | gal, all\__D, gal\__L, pyr, so4                                                           |

In [15]:
medium_rcm = list(('gthrd','glc__D','mannan','tre','thm','ribflv','trp__L','pnto__R','pydxn','btn','b12',
             'starch1200','na1','ac','cl','cys_L','gal','all__D','gal__L','pyr','so4'))
medium_rcm=[mets+"(e)" for mets in medium_rcm]
model_rcm=medium(model.copy(), medium_rcm)
for i in range(1,len(model_rcm.reactions)):
    model_rcm.reactions[i].upper_bound=1000
    model_rcm.reactions[i].lower_bound=-1000
model_rcm.objective = {model_rcm.reactions.get_by_id('Biomass_FP'): 1}
model_rcm.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
4hba[c],DM_4HBA,64.77,7,4.63%
3mop[e],EX_3mop(e),521.6,6,31.94%
acglu[e],EX_acglu(e),57.79,7,4.13%
ade[e],EX_ade(e),28.42,5,1.45%
adocbl[e],EX_adocbl(e),0.7395,72,0.54%
alaala[e],EX_alaala(e),38.38,6,2.35%
alahis[e],EX_alahis(e),12.18,9,1.12%
alaleu[e],EX_alaleu(e),57.69,9,5.30%
arg-L[e],EX_arg_L(e),37.91,6,2.32%
btn[e],EX_btn(e),0.7395,10,0.08%


In [16]:
# Post-optimization
fba_rcm = model_rcm.optimize()
fluxes = deleteNull(fba_rcm.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes),len(fluxes)/len(model.reactions)*100))
print(model_rcm.objective.value)


Active reaction : 381 which represent 36.990291% of the model
131.5139863809376


---
## mMCB

(There are also supplements mentioned in the paper but this is the base medium)
* [Bacteriological Peptome (6.5) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN): 
    * Vague, decided to go with polypeptides: <b>polypep</b>
    * Nitrogen: <b>n2</b>
* [Soy Peptome (5.0) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0044&cat=&c=UK&lang=EN) --> another vague one:
    * Stachyose: <b>stys</b>
    * Raffinose: <b>raffin</b>
    * Sucrose: <b>sucr</b>
    * Nitrogen: <b>n2</b>
* [Yeast Extract (3.0) (VWR International, Darmstadt, Germany)](https://us.vwr.com/store/product/7437401/vwr-life-science-yeast-extract-bacteriological-grade):
    * Vitamin B (same logic as RCM):
        * 1. Thiamin -> <b>thm</b>
        * 2. Riboflavin -> <b>ribflv</b>
        * 3. Niacin -> <b>trp__L</b> (made of tryptophan)
        * 5. Pantothenic Acid -> <b>pnto__R</b>
        * 6. Pyridoxine -> <b>pydxn</b>
        * 7. Biotin -> <b>btn</b>
        * 9. Folic Acid -> ... can't find anything
        * 12. Cobalamin -> <b>b12</b> (or cbl1)
* [Tryptone (2.5) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0042&c=UK&lang=EN): 
    * Tryptophan: <b>trp__L</b>
* NaCL (1.5) (VWR International, Darmstadt, Germany):
    * NaCl: <b>na1, cl</b>
* K<sub>2</sub>HPO<sub>4</sub> (1.0) (Merck International, Darmstadt, Germany):
    * Potassium: <b>k</b>
    * Phosphate: <b>p1</b>
    * Hydrogen: <b>h2</b>
* KH<sub>2</sub>PO<sub>4</sub> (1.0) (Merck International, Darmstadt, Germany):
    * Potassium: <b>k</b>
    * Phosphate: <b>p1</b>
    * Hydrogen: <b>h2</b>
* Na<sub>2</sub>SO<sub>4</sub> (2.0) (VWR):
    * Sodium: <b>na1</b>
    * Sulfate: <b>so4</b>
* MgSO<sub>4</sub>*7H<sub>2</sub>O (1.0) (Merck):
    * Magnesium: <b>mg2</b>
    * Sulfate: <b>so4</b>
* CaCl<sub>2</sub>*2H<sub>2</sub>O (0.1) (Merck):
    * Calcium Chloride: <b>ca2, cl</b>
* NH<sub>4</sub>Cl (1.0) (Merck):
    * Ammonium Chloride: <b>nh4, cl</b>
* Cysteine-HCL (0.4) (Merck):
    * Cysteine: <b>cys__L</b>
    * HCL: <b>h2, cl</b>
* NaHCO<sub>3</sub> (0.2) (VWR):
    * Sodium: <b>na1</b>
    * Bicarbonate: <b>hco3<b>
* MnSO<sub>4</sub>*H<sub>2</sub>O (0.05) (VWR):
    * Manganese: <b>mn2</b>
    * Sulfate: <b>so4</b>
* FeSO<sub>4</sub>*7H<sub>2</sub>O (0.005) (Merck):
    * Iron: <b>fe</b>
    * Sulfate: <b>so4</b>
* ZnSO<sub>4</sub>*7H<sub>2</sub>O (0.005) (VWR):
    * Zinc: <b>zn2</b>
    * Sulfate: <b>so4</b>
* Hemin (0.005) (Sigma-Aldrich, Steinheim, Germany):
    * ... contains iron
* Menadione (0.005) (S-A):
    * <b>mndn</b>
* Resazurin (0.001) (S-A):
    * Fluoro identifier



| Component                            | Concentration (g/L) | BiGG Metabolites                                |
|--------------------------------------|---------------------|-------------------------------------------------|
| Bacteriological Peptome              | 6.5                 | polypep, n2                                     |
| Soy Peptome                          | 5.0                 | stys, raffin, sucr, n2                          |
| Yeast Extract                        | 3.0                 | thm, ribflv, trp\__L, pnto\__R, pydxn, btn, b12 |
| Tryptone                             | 2.5                 | trp\__L                                         |
| NaCL                                 | 1.5                 | na1, cl                                         |
| K<sub>2</sub>PO<sub>4</sub>          | 1.0                 | k, p1, h2                                       |
| KH<sub>2</sub>PO<sub>4</sub>         | 1.0                 | k, p1, h2                                       |
| Na<sub>2</sub>SO<sub>4</sub>         | 2.0                 | na1, so4                                        |
| Mg<sub>2</sub>SO<sub>4</sub>         | 1.0                 | mg2, so4                                        |
| CaCl<sub>2</sub>*2H<sub>2</sub>O     | 0.1                 | ca2, cl                                         |
| NH<sub>4</sub>Cl                     | 1.0                 | nh4, cl                                         |
| Cysteine-HCL                         | 0.4                 | cys__L, h2, cl                                  |
| NaHCO<sub>3</sub>                    | 0.2                 | na1, hco3                                       |
| MnSO<sub>4</sub>*H<sub>2</sub>O      | 0.05                | mn2, so4                                        |
| FeSOMnSO<sub>4</sub>*7H<sub>2</sub>O | 0.005               | fe, so4                                         |
| ZnSO<sub>4</sub>*7H<sub>2</sub>O     | 0.005               | zn2, so4                                        |
| Hemin                                | 0.005               |                                                 |
| Menadione                            | 0.005               | mndn                                            |
| Resazurin                            | 0.001               |                                                 |

In [17]:
medium_mmcb = list(('polypep','stys','raffin','sucr','n2','thm','ribflv','trp_L',
                    'pnto_R','pydxn','btn','b12','na1','cl','k','p1','h','na1','h2o'
                    'so4','mg2','ca2','nh4','cys_L','cl','na1','hco3','mn2','zn2','mndnb'))

# hco3 turns into water and co2
medium_mmcb.remove('hco3')
medium_mmcb.append('co2')
medium_mmcb.append('h2o')

# alt. riboflavin
medium_mmcb.append('rbflvrd')

# Alternative form of cysteine
medium_mmcb.append('Lcyst')

# B12 referred to as cbl1,2
medium_mmcb.remove('b12')
medium_mmcb.append('cbl1')
medium_mmcb.append('cbl2')

# Glucose in bacteriological peptome
medium_mmcb.append('glc')

# Phosphate alternative
medium_mmcb.remove('p1')
medium_mmcb.append('pi')

# Remove nitrogen
medium_mmcb.remove('n2')

# L-cystine is 2 cysteine molecules
medium_mmcb.append('Lcystin')

# Fe exchanges
medium_mmcb.append('fe2')
medium_mmcb.append('fe3')

# From the paper Clemence sent; yeast extract components
# https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0236890#pone-0236890-t002
medium_mmcb.extend(['thr_L','ser_L','gly','glu_L','gln_L','asp_L','asn_L','ala_L','arg_L','arab_L','gam','man','ura','gua','ad'])

# Folic acid --> folate
medium_mmcb.append('fol')

medium_mmcb.append('cu2')

# Remove duplicates
medium_mmcb = list(set(medium_mmcb))



medium_mmcb=[mets+"(e)" for mets in medium_mmcb]
model_mmcb=medium(model.copy(), medium_mmcb)
# for i in range(1,len(model_mmcb.reactions)):
#     model_mmcb.reactions[i].upper_bound=1000
#     if model_mmcb.reactions[i].lower_bound != 0:
#         model_mmcb.reactions[i].lower_bound=-10
model_mmcb.objective = {model_mmcb.reactions.get_by_id('Biomass_FP'): 1}
model_mmcb.summary()

# Find list of all metabolites not in media
# Express in mmol/g/L
# Set as -'ve flux lower bound

Metabolite,Reaction,Flux,C-Number,C-Flux
glc-D[e],EX_glc(e),4.19,6,99.41%
hpyr[c],sink_hpyr,0.05,3,0.59%
Metabolite,Reaction,Flux,C-Number,C-Flux
but[e],EX_but(e),-0.025,4,0.40%
co2[e],EX_co2(e),-0.05,1,0.20%
glyc-R[e],EX_glyc_R(e),-0.05,3,0.59%
h[e],EX_h(e),-8.355,0,0.00%
lac-D[e],EX_lac_D(e),-8.33,3,98.81%


In [18]:
# In medium, but not used in exchange reactions
exchange_metabs = [model.exchanges[i].id for i in range(0,len(model_mmcb.exchanges))]
[metab for metab in medium_mmcb if metab not in exchange_metabs]


['EX_mn2(e)',
 'EX_h2oso4(e)',
 'EX_raffin(e)',
 'EX_stys(e)',
 'EX_zn2(e)',
 'EX_sucr(e)',
 'EX_polypep(e)',
 'EX_mndnb(e)',
 'EX_ad(e)']

In [19]:
#### Used in exchange reactions, but not present in medium
## Just metabolite
mmcb_model_medium = [metab for metab in exchange_metabs if metab not in medium_mmcb]

## BiGG metabolite name
# [raw_metab.lstrip('EX_').rstrip('(e)') for raw_metab in [metab for metab in exchange_metabs if metab not in medium_mmcb]]

## BiGG metabolite name with exchange reaction name
# [{raw_metab.lstrip('EX_').rstrip('(e)'): model_mmcb.exchanges.get_by_id(raw_metab).name} for raw_metab in [metab for metab in exchange_metabs if metab not in medium_mmcb]]

In [124]:
root_model = model.copy()
current_medium = [metab.lstrip('EX_') for metab in list(root_model.medium.keys())]

def growth_stoppers(model, metabolites):
    results = {}
    for metabolite in metabolites:
        metabolites.remove(metabolite.lstrip('EX_'))
        test_model = medium(model, metabolites)
        results[metabolite.lstrip('EX_')] = test_model.slim_optimize()
        metabolites = [metab.lstrip("EX_") for metab in metabolites]
        metabolites.append(metabolite.lstrip('EX_'))

    import pandas as pd
    results_df = pd.DataFrame(results.items(), columns=["metab","growth"]).sort_values('growth', ascending=False)

    return results_df

growth_check_df = growth_stoppers(root_model, current_medium)

# Don't take these out of the medium if you can help it:
mandatory = growth_check_df[growth_check_df['growth'] <= 10]

In [21]:
additional_metabs = [metab for metab in current_medium if metab not in medium_mmcb]
additional_metabs.remove('EX_4abz(e)')
additional_metabs.remove('EX_5oxpro(e)')
additional_metabs.remove('EX_ac(e)')
additional_metabs.remove('EX_acasp(e)')
additional_metabs.remove('EX_acgal(e)')
additional_metabs.remove('EX_acgalglcur(e)')
additional_metabs.remove('EX_acgam(e)')
additional_metabs.remove('EX_acglu(e)')
additional_metabs.remove('EX_acnam(e)')
additional_metabs.remove('EX_adocbl(e)')
# additional_metabs.remove('EX_ala_D(e)')
# additional_metabs.remove('EX_alaala(e)')
# additional_metabs.remove('EX_alaasp(e)')
# additional_metabs.remove('EX_alagln(e)')
# additional_metabs.remove('EX_alaglu(e)')
# additional_metabs.remove('EX_alagly(e)')
# additional_metabs.remove('EX_alahis(e)')
# additional_metabs.remove('EX_alaleu(e)')
# additional_metabs.remove('EX_alathr(e)')
additional_metabs.remove('EX_aso3(e)')
additional_metabs.remove('EX_aso4(e)')
additional_metabs.remove('EX_but(e)')
additional_metabs.remove('EX_butso3(e)')
additional_metabs.remove('EX_cd2(e)')
# additional_metabs.remove('EX_cellb(e)')
# additional_metabs.remove('EX_cgly(e)')
additional_metabs.remove('EX_cit(e)')
# additional_metabs.remove('EX_cobalt2(e)') #
additional_metabs.remove('EX_cps_fp(e)')
additional_metabs.remove('EX_dhor_S(e)')
additional_metabs.remove('EX_ethso3(e)')
additional_metabs.remove('EX_fe3dcit(e)')
additional_metabs.remove('EX_for(e)')
additional_metabs.remove('EX_fru(e)')
additional_metabs.remove('EX_fum(e)')
# additional_metabs.remove('EX_gal(e)')
# additional_metabs.remove('EX_galur(e)')
# additional_metabs.remove('EX_glcur(e)')
# additional_metabs.remove('EX_glyasn(e)')
# additional_metabs.remove('EX_glyasp(e)')
# additional_metabs.remove('EX_glyc3p(e)')
# additional_metabs.remove('EX_glyc_R(e)')
# additional_metabs.remove('EX_glygln(e)')
# additional_metabs.remove('EX_glyglu(e)')
# additional_metabs.remove('EX_glygly(e)')
# additional_metabs.remove('EX_glyleu(e)')
# additional_metabs.remove('EX_glymet(e)')
# additional_metabs.remove('EX_glyphe(e)')
# additional_metabs.remove('EX_glypro(e)')
# additional_metabs.remove('EX_glytyr(e)')
additional_metabs.remove('EX_gthox(e)')
additional_metabs.remove('EX_gthrd(e)')
# additional_metabs.remove('EX_h2s(e)')
additional_metabs.remove('EX_hg2(e)')
additional_metabs.remove('EX_his_L(e)')
additional_metabs.remove('EX_hxan(e)')
# additional_metabs.remove('EX_ile_L(e)')
additional_metabs.remove('EX_ins(e)')
additional_metabs.remove('EX_inulin(e)')
additional_metabs.remove('EX_isetac(e)')
additional_metabs.remove('EX_kesto(e)')
additional_metabs.remove('EX_kestopt(e)')
additional_metabs.remove('EX_kestottr(e)')
# additional_metabs.remove('EX_lac_D(e)')
# additional_metabs.remove('EX_lcts(e)')
# additional_metabs.remove('EX_leu_L(e)')
# additional_metabs.remove('EX_leugly(e)')
# additional_metabs.remove('EX_leuleu(e)')
# additional_metabs.remove('EX_lys_L(e)')
additional_metabs.remove('EX_mal_L(e)')
# additional_metabs.remove('EX_malt(e)')
# additional_metabs.remove('EX_meoh(e)')
# additional_metabs.remove('EX_met_D(e)')
# additional_metabs.remove('EX_met_L(e)')
# additional_metabs.remove('EX_metala(e)')
# additional_metabs.remove('EX_metsox_R_L(e)')
# additional_metabs.remove('EX_metsox_S_L(e)')
additional_metabs.remove('EX_mobd(e)')
additional_metabs.remove('EX_mso3(e)')
# additional_metabs.remove('EX_nac(e)')
additional_metabs.remove('EX_ncam(e)')
additional_metabs.remove('EX_o2(e)')
# additional_metabs.remove('EX_orn(e)')
# additional_metabs.remove('EX_orot(e)')
# additional_metabs.remove('EX_pb(e)')
# additional_metabs.remove('EX_pect(e)') ###
# additional_metabs.remove('EX_phe_L(e)')
additional_metabs.remove('EX_plac(e)')
# additional_metabs.remove('EX_pro_L(e)')
# additional_metabs.remove('EX_progly(e)')
additional_metabs.remove('EX_ptrc(e)')
# additional_metabs.remove('EX_pydam(e)')
# additional_metabs.remove('EX_pydx(e)')
additional_metabs.remove('EX_seln(e)')
# additional_metabs.remove('EX_so4(e)')
additional_metabs.remove('EX_spmd(e)')
# additional_metabs.remove('EX_strch1(e)')
additional_metabs.remove('EX_succ(e)')
additional_metabs.remove('EX_sulfac(e)')
# additional_metabs.remove('EX_taur(e)')
additional_metabs.remove('EX_thymd(e)')
# additional_metabs.remove('EX_tyr_L(e)')
additional_metabs.remove('EX_urate(e)')
additional_metabs.remove('EX_urea(e)')
# additional_metabs.remove('EX_val_L(e)')
additional_metabs.remove('EX_xan(e)')
additional_metabs.remove('EX_xyl_D(e)')
new_medium = [metab.lstrip('EX_') for metab in medium_mmcb + additional_metabs]
final_model = model.copy()
final_test_model = medium(final_model, new_medium)
final_test_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Lcystin[e],EX_Lcystin(e),0.5397,6,0.12%
ade[e],EX_ade(e),2.388,5,0.43%
ala-L[e],EX_ala_L(e),0.9579,3,0.10%
alaala[e],EX_alaala(e),0.996,6,0.21%
alagln[e],EX_alagln(e),2.757,8,0.79%
alahis[e],EX_alahis(e),0.9976,9,0.32%
arg-L[e],EX_arg_L(e),3.105,6,0.66%
asn-L[e],EX_asn_L(e),5.178,4,0.74%
asp-L[e],EX_asp_L(e),0.8733,4,0.12%
btn[e],EX_btn(e),0.06056,10,0.02%


In [22]:
x=[final_test_model.reactions.get_by_id(compound_id).name for compound_id in final_model.medium]
x=[compound.rstrip('(e)').rstrip("exchange").lstrip("EX_") for compound in x]
# x=[compoundname.replace("exchange", "").strip() for compoundname in x]
";".join(x)

url = "http://api.xialab.ca/mapcompounds"

payload = "{\n\t\"queryList\": \"%s;\",\n\t\"inputType\": \"name\"\n}" %";".join(x)
headers = {
    'Content-Type': "application/json",
    'cache-control': "no-cache",
    }

response = requests.request("POST", url, data=payload, headers=headers)

dict=json.loads(response.text)
mapped_compounds=pd.DataFrame(dict)

mol_weight=[]
for i in mapped_compounds["chebi_id"]:
    try:
        x= ChebiEntity(str(i))
        mol_weight.append(x.get_mass())
    except:
        mol_weight.append("NA")
        continue

mapped_compounds["mol_weight"]=mol_weight
mapped_compounds["BiGG_ID"]=[final_test_model.reactions.get_by_id(compound_id).id[3:] for compound_id in final_test_model.medium]
weights=mapped_compounds[["query", "BiGG_ID", "hit", "mol_weight"]]
# weight: g/mol
# flux: mmol/g/hr

| Component                            | Concentration (g/L) | BiGG Metabolites                                |
|--------------------------------------|---------------------|-------------------------------------------------|
| Bacteriological Peptome              | 6.5                 | polypep, n2                                     |
| Soy Peptome                          | 5.0                 | stys, raffin, sucr, n2                          |
| Yeast Extract                        | 3.0                 | thm, ribflv, trp\__L, pnto\__R, pydxn, btn, b12 |
| Tryptone                             | 2.5                 | trp\__L                                         |
| NaCL                                 | 1.5                 | na1, cl                                         |
| K<sub>2</sub>PO<sub>4</sub>          | 1.0                 | k, p1, h2                                       |
| KH<sub>2</sub>PO<sub>4</sub>         | 1.0                 | k, p1, h2                                       |
| Na<sub>2</sub>SO<sub>4</sub>         | 2.0                 | na1, so4                                        |
| Mg<sub>2</sub>SO<sub>4</sub>         | 1.0                 | mg2, so4                                        |
| CaCl<sub>2</sub>*2H<sub>2</sub>O     | 0.1                 | ca2, cl                                         |
| NH<sub>4</sub>Cl                     | 1.0                 | nh4, cl                                         |
| Cysteine-HCL                         | 0.4                 | cys__L, h2, cl                                  |
| NaHCO<sub>3</sub>                    | 0.2                 | na1, hco3                                       |
| MnSO<sub>4</sub>*H<sub>2</sub>O      | 0.05                | mn2, so4                                        |
| FeSOMnSO<sub>4</sub>*7H<sub>2</sub>O | 0.005               | fe, so4                                         |
| ZnSO<sub>4</sub>*7H<sub>2</sub>O     | 0.005               | zn2, so4                                        |
| Hemin                                | 0.005               |                                                 |
| Menadione                            | 0.005               | mndn                                            |
| Resazurin                            | 0.001               |                                                 |

In [23]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
for metab in weights.loc[weights['mol_weight'] == 'NA', 'query']:
    try:
        cid = pcp.get_synonyms(metab, 'name')[0]['CID']
        weights.loc[weights['query'] == metab, 'mol_weight'] = pcp.Compound.from_cid(cid).molecular_weight
    except:
        pass

# Still remaining
weights[weights['mol_weight'] == 'NA']

,query,BiGG_ID,hit,mol_weight
29,Fe3,fe3(e),,NA
45,exchange reaction for glycylglyci,glygly(e),,NA
48,exchange reaction for glycylphenylalai,glyphe(e),,NA
60,exchange reaction for leucylglyci,leugly(e),,NA
78,exchange reaction for pectins,pect(e),,NA
91,"starch, structure 1 (1,6-{7[1,4-Glc], 4[1,4-Gl...",strch1(e),,NA


In [73]:
weights.loc[weights['BiGG_ID'] == 'fe3(e)', 'mol_weight'] = 55.85
weights.loc[weights['BiGG_ID'] == 'glygly(e)', 'mol_weight'] = 75.07*2
weights.loc[weights['BiGG_ID'] == 'glyphe(e)', 'mol_weight'] = 75.07+165.19
weights.loc[weights['BiGG_ID'] == 'leugly(e)', 'mol_weight'] = 75.07+131.17
weights.loc[weights['BiGG_ID'] == 'pect(e)', 'mol_weight'] = 194.14
weights.loc[weights['BiGG_ID'] == 'strch1(e)', 'mol_weight'] = 359.33
# Cobalt was Co2 somehow???
weights.loc[weights['BiGG_ID'] == 'cobalt2(e)', 'mol_weight'] = 129.839


In [60]:
#ppm =mg/L =0.001 g/L

Traces_BiGG_ID=["ca2(e)" , "cl(e)", "cu2(e)", "fe2(e)", "fe3(e)", "k(e)",  "mg2(e)", "na1(e)","pb(e)"]
conc_ppm_Bact_Peptone_L37=[635, 1*10*1000, 1, 22/2, 22/2, 3.6*10*1000, 265, 1*10*1000, 0.4]
conc_ppm_Proteose_Peptone_L85=[200, 8*10*1000, 10, 42/2, 42/2, 1.4*10*1000, 340, 8*10*1000, 0.1]
conc_ppm_soya_peptone_L44=[225,0.4*10*1000, 2,90/2,90/2, 3.3*10*1000, 1530, 0.4*10*1000, 0.3]
conc_ppm_yeast_extract=[155,0.3*10*1000, 2,52/2,52/2, 7.3*10*1000, 1530, 0.3*10*1000, 0.7]
#http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN&minfo=Y

Traces=pd.DataFrame({"BiGG_ID": Traces_BiGG_ID, 
                    "conc_ppm_Bact_Peptone_L37":conc_ppm_Bact_Peptone_L37, 
                    "conc_ppm_Proteose_Peptone_L85":conc_ppm_Proteose_Peptone_L85,
                    "conc_ppm_soya_peptone_L44":conc_ppm_soya_peptone_L44, 
                    "conc_ppm_yeast_extract":conc_ppm_yeast_extract })
Traces.head()

,BiGG_ID,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract
0,ca2(e),635.0,200.0,225.0,155.0
1,cl(e),10000.0,80000.0,4000.0,3000.0
2,cu2(e),1.0,10.0,2.0,2.0
3,fe2(e),11.0,21.0,45.0,26.0
4,fe3(e),11.0,21.0,45.0,26.0


In [61]:
# AA from peptones and yeast extract
index_notinPeptones= np.where(np.isin(aa, ['asn_L(e)', 'asx_L(e)','gln_L(e)','glx_L(e)', 'his_L(e)']))
aa_BiGG_ID=aa 

aa_conc_ppm_Bact_Peptone_L37=np.array([3.92,4.99, 6.06, 1.66, 9.93, 7.71, 3.81, 3.79, 4.38, 1.58, 2.60, 5.83, 2.81, 1.25, 0.66, 0.39, 3.33])*10*1000
aa_conc_ppm_Bact_Peptone_L37=np.insert(aa_conc_ppm_Bact_Peptone_L37,index_notinPeptones[0], 0)

aa_conc_ppm_Proteose_Peptone_L85=np.array([3.81, 5.80, 5.85, 0.15, 13.78, 4.45, 4.58, 6.01, 4.61, 1.08, 4.66, 5.99, 2.18, 2.75, 0.75, 1.77, 4.11])*10*1000
aa_conc_ppm_Proteose_Peptone_L85=np.insert(aa_conc_ppm_Proteose_Peptone_L85,index_notinPeptones[0], 0)

aa_conc_ppm_soya_peptone_L44=np.array([2.57, 4.64, 7.06, 0.53, 14.71, 2.83, 2.51, 4.31, 3.77, 0.62, 0.38, 3.40, 0.67, 1.68, 0.64, 2.09, 3.65])*10*1000
aa_conc_ppm_soya_peptone_L44=np.insert(aa_conc_ppm_soya_peptone_L44,index_notinPeptones[0], 0)

aa_conc_Yeast_Extract_L21=np.array([0.91, 3.31, 7.07, 0.76, 13.49, 5.95, 4.81, 6.04, 5.40, 0.80, 3.78, 0.88, 3.42, 2.73, 0.85, 4.95, 1.00 ])*10*1000
aa_conc_Yeast_Extract_L21=np.insert(aa_conc_Yeast_Extract_L21,index_notinPeptones[0], 0)

AA=pd.DataFrame({"BiGG_ID": aa_BiGG_ID, 
                    "conc_ppm_Bact_Peptone_L37":aa_conc_ppm_Bact_Peptone_L37,
                    "conc_ppm_Proteose_Peptone_L85":aa_conc_ppm_Proteose_Peptone_L85,
                    "conc_ppm_soya_peptone_L44":aa_conc_ppm_soya_peptone_L44,
                    "conc_ppm_yeast_extract":aa_conc_Yeast_Extract_L21 })
AA.head()

,BiGG_ID,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract
0,ala_L(e),39200.0,38100.0,25700.0,9100.0
1,arg_L(e),49900.0,58000.0,46400.0,33100.0
2,asn_L(e),0.0,0.0,0.0,0.0
3,asp_L(e),60600.0,58500.0,70600.0,70700.0
4,asx_L(e),16600.0,1500.0,5300.0,7600.0


In [62]:
all_concentrations = pd.concat([Traces, AA])
weights_and_concentrations = weights.merge(all_concentrations, how="left", on="BiGG_ID")

In [68]:
weights_and_concentrations['g/L'] = weights_and_concentrations.iloc[:,4:8].sum(axis=1)/1000
weights_and_concentrations['mmol/L'] = weights_and_concentrations['g/L']/weights_and_concentrations['mol_weight']

0.7676767676767676

In [127]:
# For the sake of time, I'm doing this in excel
# Splitting up concentrations into seperate components (ie. )
weights_and_concentrations.to_csv('weights_and_concentrations.csv')
weights_and_concentrations_edited = pd.read_csv('weights_and_concentrations_edited.csv')
weights_and_concentrations_edited['g/L'] = weights_and_concentrations_edited.iloc[:,4:8].sum(axis=1)/1000
weights_and_concentrations_edited['mmol/L'] = weights_and_concentrations_edited['g/L']/weights_and_concentrations_edited['mol_weight']

In [128]:
weights_and_concentrations_edited.head()

,query,BiGG_ID,hit,mol_weight,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract,g/L,mmol/L
0,L-cysteate,Lcyst(e),Cysteic acid,169.15600,4150.0,375.0,1325.0,1900.0,7.75,0.045816
1,L-Cystine,Lcystin(e),L-Cystine,240.30256,4150.0,375.0,1325.0,1900.0,7.75,0.032251
2,Adenine,ade(e),Adenine,135.12690,1000.0,1000.0,1000.0,1000.0,4.00,0.029602
3,D-Alanine,ala_D(e),D-Alanine,89.09322,3920.0,3810.0,2570.0,910.0,11.21,0.125823
4,L-Alanine,ala_L(e),L-Alanine,89.09322,3920.0,3810.0,2570.0,910.0,11.21,0.125823


In [131]:
final_model_2 = model.copy()

for id in ["EX_"+metab for metab in list(weights_and_concentrations_edited['BiGG_ID'])]:
    final_model_2.reactions.get_by_id(id).upper_bound = 1000
    final_model_2.reactions.get_by_id(id).lower_bound = weights_and_concentrations_edited.loc[weights_and_concentrations_edited["BiGG_ID"] == id.lstrip("EX_"), "mmol/L"].values*1000
final_model_2.objective = {model_mmcb.reactions.get_by_id('Biomass_FP'): 1}
final_model_2.optimize()

<Solution infeasible at 0x7fac54e5d220>

In [126]:
mandatory['metab']

64      glyc3p(e)
119         pi(e)
110        nac(e)
106        mg2(e)
87           k(e)
141      trp_L(e)
43         cu2(e)
50         fol(e)
126       pydx(e)
41     cobalt2(e)
32         ca2(e)
39          cl(e)
131      ser_L(e)
132        so4(e)
29         btn(e)
121     pnto_R(e)
Name: metab, dtype: object

In [27]:
# Post-optimization
fba_mmcb = model_mmcb.optimize()
fluxes_mmcb = deleteNull(fba_mmcb.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes_mmcb),len(fluxes_mmcb)/len(model.reactions)*100))
print(model_mmcb.objective.value)


Active reaction : 413 which represent 40.097087% of the model
1.1150549869929224e-28


In [28]:
model_mmcb.optimize()

,fluxes,reduced_costs
26DAPLLATi,7.558565e-29,3.451266e-31
3HAD100,1.183078e-28,-2.465190e-32
3HAD120,7.170701e-29,0.000000e+00
3HAD121,4.660082e-29,0.000000e+00
3HAD140,6.560270e-29,-9.860761e-32
...,...,...
XYLt2,0.000000e+00,7.774584e-30
YUMPS,0.000000e+00,-3.298088e-31
r0502,0.000000e+00,1.156506e-30
r0839,0.000000e+00,6.606442e-15
